<center><br> <img src="https://www.iscte-iul.pt/assets/images/logo_iscte_detailed.svg" style="width: 300px;"></center><br>

<img src="https://logos-world.net/wp-content/uploads/2021/02/Expedia-Emblem.png" style="width: 300px;margin-top: 30px;" align="left">
   <div><center><h1>Projeto: Expedia Hotel Recommendations</h1></center></div>
   <div><center><h4>Processamento de Big Data  -   2º Semestre | 2022/23</h4></center></div>
   <div><center><b>Docentes:</b> João Oliveira <b>|</b> Adriano Lopes</center></div>
 <br>
    <div><center><h4>Grupo 17</h4></center> </div>
    <div><center> André Silvestre Nº104532 <b>|</b> Rita Matos Nº104936 <b>CDB1</b></center> </div>
 <br>    
 <br>
 <br>
 <br>

<style>
@import url('https://fonts.cdnfonts.com/css/avenir-next-lt-pro?styles=29974');
</style>

<div style="background: linear-gradient(to right, #075924, #067572, #0371B1, #001D45); 
            padding: 15px; color: white; border-radius: 30000px; text-align: center;">
    <center><h1 style="color: white;font-size: 30px; font-family: 'Avenir Next LT Pro', sans-serif;"><b>3 | Feature Engineering</b></h1></center>
</div>

Nesta parte iremos ter em atenção a:

- Tipos de Dados corretamente atribuidos
- Valores Omissos
- Possíveis *Outliers* 
- Remover valores discrepantes

### PySpark Session

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Expedia Project') \
    .config("spark.driver.memory", "10g") \
    .config("spark.executor.memory", "10g") \
    .config("spark.driver.host", "127.0.0.1") \
    .config("spark.sql.debug.maxToStringFields", "1000") \
    .getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1680809728276_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# 📚 Import das Bibliotecas

In [2]:
# Imports Gerais
import os
import sys

# Imports Python
import numpy as np
import pandas as pd  
import warnings
warnings.filterwarnings("ignore")

# Imports PySpark
import pyspark.sql.functions as F
from pyspark.sql.types import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# 🧮 Importar Bases de Dados (parquet)

In [3]:
# Leitura da base de dados "df-expedia-2" guardada anteriormente
# df_expedia = spark.read.parquet('df-expedia-2')
df_expedia = spark.read.parquet('s3://pdb-grupo17-bucket-as/df-expedia-2')  #AWS

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Confirmar o nº de observações e tipos de dados
print("Nº de Observações:", df_expedia.count())
df_expedia.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Nº de Observações: 37622378
root
 |-- srch_destination_id: integer (nullable = true)
 |-- date_time: string (nullable = true)
 |-- site_name: integer (nullable = true)
 |-- posa_continent: integer (nullable = true)
 |-- user_location_country: integer (nullable = true)
 |-- user_location_region: integer (nullable = true)
 |-- user_location_city: integer (nullable = true)
 |-- orig_destination_distance: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- is_mobile: integer (nullable = true)
 |-- is_package: integer (nullable = true)
 |-- channel: integer (nullable = true)
 |-- srch_ci: string (nullable = true)
 |-- srch_co: string (nullable = true)
 |-- srch_adults_cnt: integer (nullable = true)
 |-- srch_children_cnt: integer (nullable = true)
 |-- srch_rm_cnt: integer (nullable = true)
 |-- srch_destination_type_id: integer (nullable = true)
 |-- is_booking: integer (nullable = true)
 |-- cnt: integer (nullable = true)
 |-- hotel_continent: integer (nullable = true)
 

---

# 🧹 Limpeza dos Dados

### Valores Omissos

- Depois da Adição/Transformação de Variáveis, vamos voltar a verificar a existência de **valores omissos**.

In [5]:
# Ciclo for para os Valores Nulos por coluna
count_obs = df_expedia.count()
col_date = ['date_time', 'srch_ci', 'srch_co']
cols_interest = [x for x in df_expedia.columns if x not in col_date]

for cl in cols_interest:
    k = df_expedia.select(cl).filter(F.col(cl).isNull() | F.isnan(cl)).count()
    if k > 0:
        print(f'Coluna {cl}  ->  {k} valores nulos [NaN] ({k*100/count_obs:.2f}%)')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Coluna orig_destination_distance  ->  13504882 valores nulos [NaN] (35.90%)
Coluna pcaFeature_1  ->  166801 valores nulos [NaN] (0.44%)
Coluna pcaFeature_2  ->  166801 valores nulos [NaN] (0.44%)
Coluna pcaFeature_3  ->  166801 valores nulos [NaN] (0.44%)

### Possíveis *outliers*

- Verificamos ainda possíveis valores ***outliers***.

In [6]:
df_expedia.describe().toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  summary srch_destination_id  ...  srch_co_diasemana      tempo_estadia
0   count            37622378  ...           37622378           37622378
1    mean  14440.970450219813  ...  3.805820328528941  3.372074859276572
2  stddev  11065.372374915481  ...  2.164926564385831  3.092224757855745
3     min                   0  ...                  1                  0
4     max               65107  ...                  7                516

[5 rows x 41 columns]

> Após observar as medidas estatísticas, não observámos nenhum possível *outlier*.

----

#### Eliminar a *feature* `orig_destination_distance`

> Após alguma pesquisa relativa aos valores omissos na *feature* `orig_destination_distance` deparámo-nos com a confirmação do *host* da competição para esta situação. [[1]](https://www.kaggle.com/competitions/expedia-hotel-recommendations/discussion/20345)

> Assim, considerámos que deveríamos proceder à eliminação desta coluna, dado que a proporção de dados omissos é de um terço e não faria sentido eliminarmos $\sim 13.5M$ de observações do *dataset*.

In [7]:
# Eliminar a coluna orig_destination_distance
df_expedia = df_expedia.drop('orig_destination_distance')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Eliminar as observações com valores omissos de `pcaFeature_1`, `pcaFeature_2` e `pcaFeature_3`

In [8]:
# Filtrar as observações que não tenham NAs nas features pcaFeature_1, pcaFeature_2 e pcaFeature_3
df_expedia = df_expedia.filter(~(df_expedia.pcaFeature_1).isNull() & 
                               ~(df_expedia.pcaFeature_2).isNull() & 
                               ~(df_expedia.pcaFeature_3).isNull())     # Nota: '~' é usado para fazer a negação

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Recontagem das observações 37 622 378 - 37 455 577 = 166 801 obs. (0.44% das observações)
df_expedia.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

37455577

---

# 🧩 Escolha das Variáveis para os Modelos

> Aqui considerámos as **Correlações de Pearson**, realizadas na AED dos dados, com valores acima de $0.01$ porque neste contexto são as que apresentam os valores mais elevados

In [10]:
# Lista das features a usar
features_list = ['srch_destination_id', 'site_name', 'posa_continent','user_location_country', 
                 'is_package', 'srch_adults_cnt','srch_children_cnt', 'srch_destination_type_id', 
                 'is_booking','hotel_continent', 'hotel_country', 'hotel_market',
                 'pcaFeature_1', 'pcaFeature_2', 'pcaFeature_3','tempo_estadia']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Selecionar as Features
df_expedia = df_expedia.select(*features_list, 'hotel_cluster')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

---

## ✂️ Reduzir a Dimensão do Dataset

Diminuímos a dimensão do *dataset* para **30%** do seu tamanho original por questões de produtividade, tal como mencionado em aula.

In [12]:
# Total do Dataset Orignal = 37 622 378, ~ 4GB  |  Redução de 30% = 11 286 657  ~ 1,20GB
small_df_expedia = df_expedia.sample(withReplacement = False, fraction=0.3, seed=123)
small_df_expedia.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

11236139

# 💾 Guardar os dados

In [13]:
# Guardar o notebook de menor dimensão para utilização futura
# small_df_expedia.write.mode("overwrite").parquet("small-df-expedia")
small_df_expedia.write.mode("overwrite").parquet("s3://pdb-grupo17-bucket-as/small-df-expedia")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df_expedia.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- srch_destination_id: integer (nullable = true)
 |-- site_name: integer (nullable = true)
 |-- posa_continent: integer (nullable = true)
 |-- user_location_country: integer (nullable = true)
 |-- is_package: integer (nullable = true)
 |-- srch_adults_cnt: integer (nullable = true)
 |-- srch_children_cnt: integer (nullable = true)
 |-- srch_destination_type_id: integer (nullable = true)
 |-- is_booking: integer (nullable = true)
 |-- hotel_continent: integer (nullable = true)
 |-- hotel_country: integer (nullable = true)
 |-- hotel_market: integer (nullable = true)
 |-- pcaFeature_1: double (nullable = true)
 |-- pcaFeature_2: double (nullable = true)
 |-- pcaFeature_3: double (nullable = true)
 |-- tempo_estadia: integer (nullable = true)
 |-- hotel_cluster: integer (nullable = true)

In [15]:
# Guardar o dataset após a limpeza efetuada
# df_expedia.write.mode("overwrite").parquet("df-expedia-3")
df_expedia.write.mode("overwrite").parquet("s3://pdb-grupo17-bucket-as/df-expedia-3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

---

# 🔗 Referencias Bibliográficas

**[1]** Woznica, A. (2016). Expedia Hotel Recommendations - Discussion | Data leak. Kaggle.com. https://www.kaggle.com/competitions/expedia-hotel-recommendations/discussion/20345